# Working with resumable functions
Try to implement the pascal triangle, using a resumable function. Each iteration should return a line of the Pascal trianlge. (rem: a resumable function returns an iterator you need to call).

In [45]:
using ResumableFunctions

@resumable function pascal()
    a = [1]
    @yield a
    a = [1,1]
    @yield a
    while true
        a = vcat([1],a[1:end-1] .+ a[2:end],[1])
        @yield a
    end
end

pascal (generic function with 1 method)

In [47]:
p = pascal()
println("Pascal's triangle:")
for i in 1:10
    println(p())
end

Pascal's triangle:
[1]
[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]
[1, 5, 10, 10, 5, 1]
[1, 6, 15, 20, 15, 6, 1]
[1, 7, 21, 35, 35, 21, 7, 1]
[1, 8, 28, 56, 70, 56, 28, 8, 1]
[1, 9, 36, 84, 126, 126, 84, 36, 9, 1]


# small example with containers
Consider a candy machines that is continuously being monitored by a supervisor.  If the level is below a given treshold, the supervisor fills the machine up. 
* Client arrival follows an exponential distribution. 
* Look at the mean time between refills. Is this what you would expect?
* What happend when the amount of candy varies? Is this still what you would expect?

In [130]:
# dependencies
using SimJulia
using Distributions
using Logging
Logging.disable_logging(LogLevel(1000)); # to avoid logmessages

In [143]:
# processes

# process of client arrivals
@resumable function clientarrival(sim::Simulation,c::Container,amount=[2])
    while true
        @yield timeout(sim,rand(Distributions.Exponential(1)))
        @info "client arrives on time $(sim.time)"
        @process getcandy(sim,c,amount)
    end
end

# process of getting candy
@resumable function getcandy(sim::Simulation,c::Container,amount=[2])
    @yield get(c,rand(amount))
end

# monitor
@resumable function monitor(sim::Simulation,c::Container,trefills::Array)
    while true
        @yield timeout(sim,1)
        if c.level <= c.capacity*0.10
            #println("Oh no, almost empty! Time to refill... $(sim.time)")
            @logmsg LogLevel(3000) "Oh no, almost empty! Time to refill... $(sim.time)"
            push!(trefills,sim.time)
            @yield timeout(sim,2) # temps de replissage
            c.level = c.capacity # actual refill
            @logmsg LogLevel(3000) "Finished refilling $(sim.time)"
        end
    end
end

monitor (generic function with 1 method)

In [144]:
function runsim(amount=[2])
    sim = Simulation()
    candymachine = Resource(sim,100,level=100)
    trefills = [0]
    @process clientarrival(sim,candymachine,amount)
    @process monitor(sim,candymachine,trefills)
    run(sim,1000)
    return mean(trefills[2:end] .- trefills[1:end-1])
end

runsim (generic function with 2 methods)

If on average, we have 1 client/timeunit and that clients takes 2 candies it would take 50 timeunits to be depleted. However, we refill as soon as we are below 10% of the capacity, which should lead to a mean time between refills of about 45 timeunits.

In [149]:
Logging.disable_logging(LogLevel(3000))
runsim()

49.95

If on average, we have 1 client/timeunit and that clients takes between 1 and 10 candies, we have on average 5.5 candies per timeunit.  It would take 16 timeunits to be depleted. This result is what we find below. Differences are due to the fact that the reloading time is important with respect to the arrival rate of clients.

In [156]:
runsim(collect(1:10))

17.963636363636365

# Small example(s) with stores
When modeling physical things such as cables, RF propagation, etc. it better encapsulation to keep this propagation mechanism outside of the sending and receiving processes.

Consider the following:
* a sender sends messages on a regular interval (e.g. every 5 minutes)
* a receiver is listening on a permanent basis for new messages
* the transfer between both of them is not instantaneous, but takes some time. To model this, you can store (hint: use a `::Store`) the messages on the cable for later reception.


In [1]:
using SimJulia
const SIM_DURATION = 100.0

struct Cable
  sim::Simulation
  delay::Float64
  store::Store{String}
  function Cable(sim::Simulation, delay::Float64)
    cable = new(sim, delay, Store{String}(sim) )
    return cable
  end
end


@resumable function sender(sim::Simulation, cable::Cable)
    while true
        @yield timeout(sim,5.0)
        @process latency(sim,cable,"Sender send this at $(sim.time)")
    end
end

@resumable function latency(sim::Simulation, cable::Cable, value::String)
    @yield timeout(sim, cable.delay)
    @yield SimJulia.put(cable.store, value)
end

@resumable function receiver(sim::Simulation, cable::Cable)
    while true
        msg = @yield get(cable.store)
        println("Received this at $(sim.time) while $msg")
    end
end

receiver (generic function with 1 method)

In [2]:
println("Event latency")
sim = Simulation()
cable = Cable(sim, 10.0)
@process sender(sim, cable)
@process receiver(sim,cable)

run(sim, SIM_DURATION)

Event latency
Received this at 15.0 while Sender send this at 5.0
Received this at 20.0 while Sender send this at 10.0
Received this at 25.0 while Sender send this at 15.0
Received this at 30.0 while Sender send this at 20.0
Received this at 35.0 while Sender send this at 25.0
Received this at 40.0 while Sender send this at 30.0
Received this at 45.0 while Sender send this at 35.0
Received this at 50.0 while Sender send this at 40.0
Received this at 55.0 while Sender send this at 45.0
Received this at 60.0 while Sender send this at 50.0
Received this at 65.0 while Sender send this at 55.0
Received this at 70.0 while Sender send this at 60.0
Received this at 75.0 while Sender send this at 65.0
Received this at 80.0 while Sender send this at 70.0
Received this at 85.0 while Sender send this at 75.0
Received this at 90.0 while Sender send this at 80.0
Received this at 95.0 while Sender send this at 85.0


## Another store example: 
suppose you have two machines, each producing a different product (with different production times). The assembly of a third product requires both of these. Analyse this simple case to find bottlenecks is the proces

In [158]:
struct Product
    kind::String
    id::Int
end

@resumable function machine(sim::Simulation,s::Store,prodname::String,dur)
    i = 1
    while true
        @yield timeout(sim,dur)
        @yield put(s,Product(prodname,i))
        i+=1
    end
end

@resumable function combiner(sim::Simulation,s1::Store,s2::Store)
    while true
        #@yield timeout(sim,3)
        println("trying to combine on time $(sim.time)")
        r1 = get(s1)
        r2 = get(s1)
        r3 = get(s2)
        res = @yield r1 & r2 & r3
        println("making a product on time $(sim.time)")
        @yield timeout(sim,2)
    end
end 

combiner (generic function with 1 method)

In [159]:
sim = Simulation()
boltstore = Store{Product}(sim)
nutstore  = Store{Product}(sim)
pbolt = @process machine(sim,boltstore,"bolt",1)
pnut  = @process machine(sim,nutstore,"nut",4)
@process combiner(sim,boltstore,nutstore)
run(sim,30)

trying to combine on time 0.0
making a product on time 4.0
trying to combine on time 6.0
making a product on time 8.0
trying to combine on time 10.0
making a product on time 12.0
trying to combine on time 14.0
making a product on time 16.0
trying to combine on time 18.0
making a product on time 20.0
trying to combine on time 22.0
making a product on time 24.0
trying to combine on time 26.0
making a product on time 28.0


In [161]:
boltstore.items

Set(Product[Product("bolt", 19), Product("bolt", 24), Product("bolt", 12), Product("bolt", 4), Product("bolt", 18), Product("bolt", 6), Product("bolt", 22), Product("bolt", 26), Product("bolt", 27), Product("bolt", 28), Product("bolt", 10), Product("bolt", 29), Product("bolt", 15), Product("bolt", 21), Product("bolt", 20)])